In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [20]:
data = """
 A    B   C      D                             E
    0  0    0.0    False   1970-01-01 00:00:00.000000001
"""

df = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['E'])

df

,A,B,C,D,E
0,0,0,0.0,False,1970-01-01 00:00:00.000000001


In [13]:
data = """
  Integers  Numeric  Floats   Strings  Booleans      Dates   Dates_Right
    0         0      NaN     0.0        r     False   1970-01-01  1970-01-01
    1         0      0.0     0.0        r     False   1970-01-01  1970-01-01
"""

right = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['Dates', 'Dates_Right'])

right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,NaN,0.0,r,False,1970-01-01,1970-01-01
1,0,0.0,0.0,r,False,1970-01-01,1970-01-01


In [21]:
df.to_dict('records')

[{'A': 0,
  'B': 0,
  'C': 0.0,
  'D': False,
  'E': Timestamp('1970-01-01 00:00:00.000000001')}]